**Configuração e Autenticação**

In [ ]:
import os

from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery

import pandas as pd

import requests
import base64

**Conexão Big Query**

In [ ]:
project_id = 'case-gb-elcio'
dataset_id = 'spotify'

In [ ]:
client = bigquery.Client(project=project_id)

In [ ]:
def upload_to_bq(df, table_name):
    job_config = bigquery.LoadJobConfig(
        autodetect=True
    )

    job = client.load_table_from_dataframe(
        df, f'{project_id}.{dataset_id}.{table_name}', job_config=job_config
    )
    job.result()

    print(f"Tabela {table_name} carregada com sucesso!")

**Conectando a API do Spotify e buscando token**

In [ ]:
client_id = os.environ['client_id']
client_secret = os.environ['client_secret']

client_creds = f"{client_id}:{client_secret}"
client_creds_b64 = base64.b64encode(client_creds.encode()).decode()

auth_url = "https://accounts.spotify.com/api/token"

auth_headers = {
    "Authorization": f"Basic {client_creds_b64}",
    "Content-Type": "application/x-www-form-urlencoded"
}

auth_data = {
    "grant_type": "client_credentials"
}

auth_response = requests.post(auth_url, headers=auth_headers, data=auth_data)

if auth_response.status_code == 200:
    access_token = auth_response.json()["access_token"]
else:
    print("Erro na autenticação")

**Função que pesquisa pelo episodio**

In [ ]:
def pesquisar_spotify(id, offset=0)

    search_url = f"https://api.spotify.com/v1/shows/{id}/episodes?offset={offset}&limit=50"


    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.get(search_url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro na pesquisa: {response.status_code}")
        return None

**Função que trás todos os resultados não nulos dos episodios de data hackers**

In [ ]:
def extrair_dados_episodios(lista_ids):
  episodios = []
  for id in lista_ids:
    offset = 0
    while True:
      resultado = pesquisar_spotify(id, offset)

      if resultado and resultado['items']:
        for item in resultado['items']:
          episodio = {
            'id': item['id'],
            'nome': item['name'],
            'descricao': item['description'],
            'data_lancamento': item['release_date'],
            'duracao_ms': item['duration_ms'],
            'idioma' : item['language'],
            'explicito': item['explicit'],
            'tipo': item['type']
          }
          episodios.append(episodio)

        if resultado['next']:
          offset += len(resultado['items'])
        else:
          break
      else:
        print(f"Erro ao buscar episódios do ID: {id}")
        break

  return episodios

**Criando pdDF de todos os episódios encontrados e salvando no BigQuery**

In [ ]:
ids = ["1oMIHOXsrLFENAeM743g93"]
tabela_episodios = extrair_dados_episodios(ids)
df_episodes_data_hackers = pd.DataFrame(tabela_episodios)

In [ ]:
upload_to_bq(df_episodes_data_hackers, 'tb_episodes_data_hackers')

**Criando pdDF com os resultados relacionados a Grupo Boticário e salvando a tabela no BigQuery**

In [ ]:
palavras_chave = ['Grupo Boticário', 'Grupo Boticario', 'Boticário', 'Boticario', 'O Boticário', 'O Boticario']

palavras_chave_lower = [palavra.lower() for palavra in palavras_chave]

regex_palavras_chave = '|'.join(palavras_chave_lower)

df_episodes_gb_data_hackers = df_episodes_data_hackers[df_episodes_data_hackers['nome'].str.contains(regex_palavras_chave, case=False)]

In [ ]:
upload_to_bq(df_episodes_gb_data_hackers, 'tb_episodes_gb_data_hackers')